<img src='../__docs/__all/notebook_rules.png' />

## Full workflow

* create file name list vs time stamp -> [create_list_of_file_name_vs_time_stamp.ipynb](../create_list_of_file_name_vs_time_stamp.ipynb)
* normalize by first image -> [normalization.ipynb](../normalization.ipynb)
* crop and create stack of images to work on -> [rotate_and_crop_images.ipynb](../rotate_and_crop_images.ipynb)
* create profile -> [profile.ipynb](../profile.ipynb)

In [ ]:
import os
import pandas as pd
import numpy as np

import plotly.graph_objects as go
import plotly.offline as pyo
pyo.init_notebook_mode()
import plotly.express as px

import plotly.io as pio
pio.renderers.default = 'iframe'

import re

# User input here

In [ ]:
ipts_root_folder = "/Users/j35/HFIR/CG1D/IPTS-32328/"
assert os.path.exists(ipts_root_folder)

data_folder = "2024_03_07/2024_03_07_batteries_2D_cycling_timestamp_infos.txt"

profile_file = os.path.join(ipts_root_folder, "2024_03_07/profile_sample1.txt")
assert os.path.exists(profile_file)

images_combine_by = 3

# Load file_name_vs_time_stamp 

We need to match each file to its time stamp

In [ ]:
file_name_vs_time_stamp = os.path.join(ipts_root_folder, data_folder)
assert os.path.exists(file_name_vs_time_stamp)

Load ASCII

In [ ]:
pd_data = pd.read_csv(file_name_vs_time_stamp)

In [ ]:
pd_data

Extract file names and time offset, and keep only base file names

In [ ]:
list_files = list(pd_data['#filename'])
short_list_files = [os.path.basename(_file) for _file in list_files]
list_timeoffset = list(pd_data[' timeoffset(s)'])

if data have been combined, use timeoffset accordingly

In [ ]:
combine_factor = images_combine_by
combined_list_timeoffset_s = list_timeoffset[0::combine_factor]
combined_list_timeoffset_mn = [_t/60 for _t in combined_list_timeoffset_s]
combined_list_timeoffset_hr = [_t/60 for _t in combined_list_timeoffset_mn]

fig = go.Figure()
fig.add_trace(go.Scatter(y=combined_list_timeoffset_hr,
                        mode='markers'))
fig.update_layout(title="Time offset vs file index",
                  xaxis_title="file index",
                  yaxis_title="time offset (hr)",
                  )
fig.show()

Create correct time offset axis in hours

# User input here! 

In [ ]:
max_hour = 16

In [ ]:
def get_closest_index(array=None, value=None):
    idx = (np.abs(np.array(array) - value)).argmin()
    return idx

list_hr_to_display = np.arange(max_hour)
list_index_hr_to_display = []
for _hr in list_hr_to_display:
    _index = get_closest_index(array=combined_list_timeoffset_hr,
                               value=_hr)
    list_index_hr_to_display.append(_index)

tick_values = list_index_hr_to_display
tick_text = [str(_hr) for _hr in list_hr_to_display]


Checking that they are correctly sorted

In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter(y=list_timeoffset,
                        mode='markers'))
fig.update_layout(title="Time offset vs file index",
                  xaxis_title="file index",
                  yaxis_title="time offset (s)",
                  )
fig.show()

# Load profile 

working with sample 1 of 24_03_07

In [ ]:
with open(profile_file, 'r') as f:
    content = f.readlines()

In [ ]:
# extract number of files (nbr of colons = number of files + 1)

pattern = r"\#\w*\s{1}\w*\s{1}\w*\s{1}\((\d*)\s{1}\w*\)"

list_of_files_row = 0
nbr_files = 0
for index, line in enumerate(content):
    if line.startswith("#List of files "):
        list_of_files_row = index + 1
        m = re.match(pattern, line)
        if m:
            print(f"list of files: {m.group(1)}")
            nbr_files = int(m.group(1))
# print(f"{list_of_files_row =}")

In [ ]:
# extract name of files
list_of_files = list(pd.read_csv(profile_file, skiprows=6, nrows=nbr_files, names=['file name']))

In [ ]:
# extract data
column_names = ['pixel offset']
for _index in np.arange(nbr_files):
    column_names.append(f"file#{_index}")
pd_data = pd.read_csv(profile_file, names=column_names, comment='#')

nbr_pixel = len(pd_data['pixel offset'])

In [ ]:
# make 2D array
data_2d = np.zeros((nbr_pixel, nbr_files))
for _row in np.arange(nbr_pixel):
    for _col in np.arange(nbr_files):
        _col_data = pd_data[column_names[_col+1]][_row]
        data_2d[_row, _col] = _col_data

In [ ]:
fig = go.Figure(data = go.Contour(z=data_2d))
fig.update_layout(title=os.path.basename(profile_file),
                  xaxis = dict(tickmode='array',
                               tickvals=tick_values,
                               ticktext=tick_text),
                  xaxis_title="Time offset (hours)",
                  yaxis_title="Pixel counts normalized by first image")



In [ ]:
fig = go.Figure(data = go.Heatmap(z=data_2d, zsmooth='best'))
fig.update_layout(title=os.path.basename(profile_file),
                  xaxis = dict(tickmode='array',
                               tickvals=tick_values,
                               ticktext=tick_text),
                  xaxis_title="Time offset (hours)",
                  yaxis_title="Pixel counts normalized by first image")
                  

Transmission normalized by first image:
 * value of 1 means no change
 * value > 1 means transmission compare to the first image is higher
 * value < 1 means transmission compare to the first image is lower